In [33]:
import pandas as pd

filename = '../big_a_data/all.csv'
df = pd.read_csv(filename, converters={'code': str})
#df = df[df['code'].isin(['600660', '600036', '600750', '600941', '601318', '001965', '000999', '601601', '601225', '000538'])]
df = df[df['股息率_2024'] > 2].copy()
df = df[~df['code'].str.startswith(('300', '688', '8'))].copy() #去掉创业板, 科板, 北交
print('Done', len(df))

DS_NAME = 'all'
#DS_NAME = '自选'
df = df[['code', 'name', '股息率_2024']].copy()

Done 900


In [34]:
str_today = '2025-05-29'
DURATION = 330
DAYS_PRE = 2
DAYS_NEXT = 5
ZHANG = 0.5
cols_pre = [f'd-{i}' for i in range(DAYS_PRE, 0, -1)]
cols_next = [f'd{i}' for i in range(1, DAYS_NEXT + 1)]
filename_output = f'涨买_前{DAYS_PRE}+日线绿_今日线红_收盘价买入_后{DAYS_NEXT}日_涨{ZHANG}_{str_today}_{DS_NAME}'
df_out = pd.DataFrame(columns=['code', 'select', 'get', 'rate', 'profit_min'])

i = 0
for code in df['code']:
    i += 1
    print(i, code)
    df_process = pd.read_csv(f'../big_a_ds/process_daily_2nd/{code}.csv')
    if df_process.iloc[-1]['日期'] != str_today: continue #停牌票
    if len(df_process) < DURATION + DAYS_PRE + DAYS_NEXT + DAYS_NEXT: continue #不够数量
    
    #prepare data
    df_select = df_process[-DURATION - DAYS_NEXT : -DAYS_NEXT].copy()
    df_select[f'后{DAYS_NEXT}日最大涨幅'] = df_select[cols_next].max(axis=1)
    
    #buy condition
    df_select = df_select[df_select['收盘'] > df_select['开盘']]
    #df_select = df_select[df_select['涨跌幅'] > 0]
    df_select = df_select[df_select[cols_pre].eq('green').all(axis=1)]
    
    #get
    df_get = df_select[df_select[f'后{DAYS_NEXT}日最大涨幅'] >= ZHANG]
    
    #record
    if len(df_get) > 0:
        new_row = {'code': code, 
                   'select': len(df_select), 
                   'get': len(df_get), 
                   'rate': len(df_get) / len(df_select) * 100, 
                   'profit_min': df_get[f'后{DAYS_NEXT}日最大涨幅'].min()}
        df_out.loc[len(df_out)] = new_row

df_out = pd.merge(df, df_out, on='code', how='inner')
print('\r\nDone', len(df_out))
#print(df_out)
df_out['rate'] = df_out['rate'].round(2)
df_out.sort_values(by=['rate', 'select'], ascending=False, inplace=True)
df_out.to_csv(filename_output + '.csv', index=False)

1 000001
2 000019
3 000026
4 000027
5 000030
6 000039
7 000048
8 000088
9 000090
10 000156
11 000157
12 000333
13 000338
14 000401
15 000403
16 000404
17 000411
18 000419
19 000423
20 000425
21 000429
22 000498
23 000501
24 000513
25 000521
26 000528
27 000529
28 000537
29 000538
30 000543
31 000544
32 000550
33 000552
34 000559
35 000563
36 000568
37 000581
38 000582
39 000589
40 000591
41 000596
42 000598
43 000601
44 000623
45 000625
46 000630
47 000650
48 000651
49 000655
50 000661
51 000672
52 000683
53 000685
54 000690
55 000700
56 000707
57 000708
58 000718
59 000719
60 000726
61 000728
62 000731
63 000739
64 000755
65 000776
66 000783
67 000786
68 000789
69 000791
70 000807
71 000811
72 000823
73 000828
74 000830
75 000848
76 000850
77 000858
78 000875
79 000883
80 000885
81 000895
82 000897
83 000899
84 000900
85 000902
86 000906
87 000910
88 000913
89 000915
90 000920
91 000921
92 000923
93 000928
94 000932
95 000933
96 000937
97 000951
98 000960
99 000963
100 000967
101 0009

In [35]:
import pandas as pd

df_temp = pd.read_csv(filename_output + '.csv', converters={'code': str})
#df_temp = df_temp[df_temp['股息率_2024'] > 2]
df_temp = df_temp[df_temp['select'] > 33]
df_temp = df_temp[df_temp['rate'] > 90]
df_temp.sort_values(by=['rate', 'select'], ascending=False, inplace=True)
print(df_temp)
print(len(df_temp))
df_temp.to_csv(filename_output + '_关注.csv', index=False)

       code  name  股息率_2024  select  get    rate  profit_min
0    002353  杰瑞股份      2.40      43   43  100.00        0.70
1    600426  华鲁恒升      2.93      41   41  100.00        0.61
2    603976  正川股份      2.02      41   41  100.00        0.83
3    301290  东星医疗      2.41      39   39  100.00        0.86
4    603966  法兰泰克      2.34      37   37  100.00        0.95
..      ...   ...       ...     ...  ...     ...         ...
293  601619  嘉泽新能      3.07      41   37   90.24        0.58
294  603170  宝立食品      2.96      41   37   90.24        1.08
295  603939  益丰药房      2.39      41   37   90.24        0.70
296  605066  天正电气      4.94      41   37   90.24        0.57
297  603801  志邦家居      5.71      51   46   90.20        0.68

[279 rows x 7 columns]
279
